<a href="https://colab.research.google.com/github/duypham01/PDE_poisson_independent_time/blob/main/Poisson_independant_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
import numpy as np
import os
import cv2
import csv
import matplotlib.pyplot as plt
import math
import sys
import scipy.io
from scipy.interpolate import griddata
import time
from itertools import product, combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [58]:
%tensorflow_version 1.x

In [59]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [150]:
class PDENet:
    # Init
    def __init__(self, xb, yb, ub, x, y, layers):
        
        # Xb = np.concatenate([xb, yb, ub], 1)
        # X = np.concatenate([x, y], 1)
        self.dim = 2
        
        self.xb = xb
        self.yb = yb
        self.ub = ub
        self.x = x
        self.y = y
        self.layers = layers
        
        # Initialize NN
        self.weights, self.biases = self.init_NN(layers)
        
        # Initialize parameters
        # self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        # self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.xb_tf = tf.placeholder(tf.float32, shape=[None, 1, 1])
        self.yb_tf = tf.placeholder(tf.float32, shape=[None, 1, 1])
        self.ub_tf = tf.placeholder(tf.float32, shape=[None, 1])

        self.x_tf = tf.placeholder(tf.float32, shape=[None, 1, 1])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, 1, 1])

        self.ub_pred, _ , _ = self.net_u(self.xb_tf, self.yb_tf)
        _ , self.f_u_pred= self.net_f_u(self.x_tf, self.y_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.ub_tf - self.ub_pred)) + \
                    tf.reduce_mean(tf.square(self.f_u_pred))
                    
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})        
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)                    
        self.saver = tf.train.Saver()
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def init_NN(self, layers):
        dim = self.dim
        weights = []
        biases = []
        num_layers = len(layers)
        W = self.xavier_init(size=[layers[0], dim])
        b = tf.Variable(tf.zeros([layers[0],1], dtype=tf.float32), dtype=tf.float32)
        weights.append(W)  
        biases.append(b)
        for l in range(1,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l]])
            b = tf.Variable(tf.zeros([layers[l],1], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)  
        W = self.xavier_init(size=[1, layers[-1]])
        b = tf.Variable(tf.zeros([1,1], dtype=tf.float32), dtype=tf.float32)
        weights.append(W)  
        biases.append(b) 
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def net_nn(self, X, weights, biases):
        num_layers = len(self.layers)
        S = X
        for l in range(0,num_layers-1):
            W = weights[l]
            b = biases[l]
            S = tf.tanh(tf.add(tf.matmul(W, S), b))
        W = weights[-1]
        b = biases[-1]
        S = tf.add(tf.matmul(W, S), b)
        return S

    def net_u(self, x, y):
        u = self.net_nn(tf.concat([x,y], 1), self.weights, self.biases)
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        return u, u_x, u_y
        
    def net_f_u(self, x, y):
        u, u_x, u_y = self.net_u(x, y)

        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]

        f_u = (-1.0)*(u_xx + u_yy) - 2.0*(np.pi)**2*tf.math.sin(np.pi*x)*tf.math.sin(np.pi*y)
        
        return u, f_u
    
    def callback(self, loss):
        print('Loss: %.3e' % (loss))
      
    def train(self, nIter): 

        tf_dict = {self.xb_tf: self.xb, self.yb_tf: self.yb, self.ub_tf: self.ub,
                   self.x_tf: self.x, self.y_tf: self.y}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            # if it % 10 == 0:
            elapsed = time.time() - start_time
            loss_value = self.sess.run(self.loss, tf_dict)
            print('It: %d, Loss: %.3e, Time: %.2f' % 
                    (it, loss_value, elapsed))
            start_time = time.time()
            
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
        
        # self.saver.save(self.sess, 'my-model.ckpt')
    
    def predict(self, x, y):
        
        tf_dict = {self.xb_tf: x, self.yb_tf: y}
        
        u = self.sess.run(self.ub_pred, tf_dict)

        # tf_dict = {self.x_tf: x, self.y_tf: y}
        
        # f_u = self.sess.run(self.f_u_pred, tf_dict)
        
        return u

In [95]:
def splitData(data):
    x = []
    y = []
    xb = []
    yb = []
    ub = []
    for row in data:
        if (row[0] == 0. or row[1] == 0. or row[0] == 1. or row[1] == 1.):
            xb.append(row[0])
            yb.append(row[1])
            ub.append(0.0)
        else:
            x.append(row[0])
            y.append(row[1])
    return np.array(x), np.array(y), np.array(xb), np.array(yb), np.array(ub)

In [237]:
dataset = np.genfromtxt('modified_rectangle_data.txt', delimiter=',')
x,y,xb,yb,ub = splitData(dataset)
x = x.reshape((x.size, 1, 1))
y = y.reshape((y.size, 1, 1))
xb = xb.reshape((xb.size, 1, 1))
yb = yb.reshape((yb.size, 1, 1))
ub = ub.reshape((ub.size, 1))
layers = [32, 32, 32, 32, 32]



In [238]:
model = PDENet(xb, yb, ub, x, y, layers)
start_time = time.time()                
model.train(500)
elapsed = time.time() - start_time                
print('Training time: %.4f' % (elapsed))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

It: 0, Loss: 1.034e+02, Time: 2.94
It: 1, Loss: 1.027e+02, Time: 0.19
It: 2, Loss: 1.021e+02, Time: 0.20
It: 3, Loss: 1.015e+02, Time: 0.19
It: 4, Loss: 1.010e+02, Time: 0.19
It: 5, Loss: 1.005e+02, Time: 0.18
It: 6, Loss: 9.992e+01, Time: 0.19
It: 7, Loss: 9.938e+01, Time: 0.18
It: 8, Loss: 9.881e+01, Time: 0.17
It: 9, Loss: 9.821e+01, Time: 0.20
It: 10, Loss: 9.757e+01, Time: 0.19
It: 11, Loss: 9.688e+01, Time: 0.19
It: 12, Loss: 9.613e+01, Time: 0.19
It: 13, Loss: 9.531e+01, Time: 0.19
It: 14, Loss: 9.442e+01, Time: 0.20
It: 15, Loss: 9.345e+01, Time: 0.19
It: 16, Loss: 9.239e+01, Time: 0.20
It: 17, Loss: 9.124e+01, Time: 0.19
It: 18, Loss: 8.999e+01, Time: 0.18
It: 19, Loss: 8.863e+01, Time: 0.19
It: 20, Loss: 8.716e+01, Time: 0.18
It: 21, Loss: 8.557e+01, Time: 0.17
It: 22, Loss: 8.386e+01, Time: 0.21
It: 23, Loss: 8.202e+01, Time: 0.19
It: 24, Loss: 8.006e+01, Time: 0.17
It: 25, Loss: 7.79

In [239]:
def exact_solution(x, y):
    u = tf.math.sin(np.pi*x)*tf.math.sin(np.pi*y)
    return u

In [240]:
datasetTest = np.genfromtxt('test_data.txt', delimiter=',')
x,y,xb,yb,ub = splitData(datasetTest)
x = x.reshape((x.size, 1, 1))
y = y.reshape((y.size, 1, 1))
xb = xb.reshape((xb.size, 1, 1))
yb = yb.reshape((yb.size, 1, 1))
ub = ub.reshape((ub.size, 1))


In [241]:
# ub_e = exact_solution(xb, yb)
u_e = exact_solution(x, y)

In [242]:
u_pred = model.predict(x, y)
ub_pred = model.predict(xb, yb)

In [243]:
LossL2_u = tf.reduce_mean(tf.square(u_pred - u_e))
with tf.Session() as sess:
    result = LossL2_u.eval()

    print(result) 

    print(type(result))

4.519140230729397e-08
<class 'numpy.float64'>


In [244]:
LossL2_u= tf.math.sqrt(LossL2_u)
with tf.Session() as sess:
    result = LossL2_u.eval()

    print(result) 

    print(type(result))

0.00021258269522069281
<class 'numpy.float64'>


In [245]:
LossL2 = tf.reduce_mean(tf.square(u_pred - u_e)) + tf.reduce_mean(tf.square(ub_pred - ub))
with tf.Session() as sess:
    result = LossL2.eval()

    print(result) 

    print(type(result))

4.604994209726471e-07
<class 'numpy.float64'>


In [246]:
LossL2= tf.math.sqrt(LossL2_u)
with tf.Session() as sess:
    result = LossL2.eval()

    print(result) 

    print(type(result))

0.014580215883885013
<class 'numpy.float64'>


In [ ]:
newModel = PDENet(xb, yb, ub, x, y, layers)

In [ ]:
tf.reset_default_graph()
newModel.saver.restore(newModel.sess, 'my-model.ckpt')